In [ ]:
#load "Common.fsx"
#r "nuget: FSharp.Stats, 0.4.6"
#r "nuget: Plotly.NET, 3.0.0"
#r "nuget: Plotly.NET.Interactive, 3.0.0"
#r "nuget: Numpy, 3.10.1.29"

open Common
open Common.NumpyUtils
open FSharp.Stats
open FSharp.Stats.Distributions
open Plotly.NET
open Numpy

Installed Packages FSharp.Stats, 0.4.6 Numpy, 3.10.1.29 Plotly.NET, 3.0.0 Plotly.NET.Interactive, 3.0.0

Loading extensions from `Plotly.NET.Interactive.dll`


**Reconstructing the posterior distribution via sampling.**

**Estimate posterior distribution with the Grid Approximation method**

- Choose n points to estimate
- Compute the likelihoods over a range of probabilities (parameter values) (binomial distribution)
- Adjust for the given prior (uniform, heavyside, etc ...)
- Standardize
- The resulting posterior distribution of parameter values will have a posterior probability associated to each value.
- Perform a weighted sampling from this posterior distribution.

In [ ]:
let xs = 
    let nTrials = 10
    let nSuccesses = 1
    let probas = np.linspace(0, 1, 1000) |> toSharp
    let prior = [for _ in 0. .. ((float probas.Length) - 1.) -> 1.]
    let likelihood = [for p in probas -> (Discrete.binomial p nTrials).PDF nSuccesses]
    let posterior = rethinking.computePosterior likelihood prior

    [100; 1000; 10000; 100000]
    |> List.map (fun xs -> 
        np.random.choice(a=toNDarray probas, size=[|xs|], p=toNDarray posterior) 
        |> toSharp
        |> fun xs ->
            let traceName = $"# Samples: {xs.Length}"
            Chart.Histogram(xs, Name=traceName))
    |> Chart.combine
    |> Chart.withXAxisStyle("Proportion of water (parameter p)", MinMax=(0., 1.))
    |> Chart.withYAxisStyle("Counts")
    |> fun samplingFig ->
        [samplingFig; 
            rethinking.binomialMultiGridApproximationFig (fun x -> 1.) [probas.Length] nTrials nSuccesses]
    |> Chart.Grid(2, 1)
    |> Chart.withSize(2000, 500)
    |> Chart.withTitle($"Binomial Distribution (nTrials: {nTrials}, nSucesses: {nSuccesses})")
    |> Chart.withDescription(ChartDescription.create "<h1>Reconstructing the posterior distribution via sampling</h1>" "")

In [ ]:
xs.Display()

<!-- Plotly chart will be drawn inside this DIV -->